In [ ]:
mask = df_new.unique_operator_id_count > 2

df_new[mask] 

df_count = phone_trunc_grouped_filtered.groupby(['phone_trunc']).count()

phone_trunc_mask = df_count.operator_id > 2

df_count[phone_trunc_mask]


In [ ]:
# view data

# reset les indexes après un group by -> Pas besoin de multi indexes et bug si filtre avec multi
df_carpool_pt_index = phone_trunc_grouped_filtered.set_index(['phone_trunc'])
df_linear_view = df_carpool_pt_index.sort_index()[['_id', 'is_driver', 'datetime','duration', 'operator_id', 'rpc_uuid']]

df_linear_view

In [ ]:
from datetime import timedelta

import pandas as pd

def has_overalp(row1, row2):
  start1 = pd.to_datetime(row1['datetime'])
  duration1 = pd.to_timedelta(row1['duration'], unit='s')
  
  start2 = row2['datetime']
  duration2 = pd.to_timedelta(row2['duration'], unit='s')

  end1 = start1 + duration1
  end2 = start2 + duration2

  overlap_start = max(start1, start2)
  overlap_end = min(end1, end2)
  return max(0, (overlap_end - overlap_start).total_seconds())


def tag_overlap_rows_in_group(df):
  for i in range(len(df)):
    no_overlap_count = 0
    for j in range(len(df)):
      overlap_duration = has_overalp(df.iloc[i], df.iloc[j])
      if overlap_duration == 0:
        no_overlap_count =+1
    if no_overlap_count == (len(df) -1):
      df.loc[df.index[i], 'has_overlap'] = False
    else:
      df.loc[df.index[i], 'has_overlap'] = True
  return df

df_phone_trunc_with_overlap = phone_trunc_grouped_filtered.groupby(['phone_trunc']).apply(lambda df: tag_overlap_rows_in_group(df))
remove_overlap_mask = df_phone_trunc_with_overlap.has_overlap == True
df_only_grouped_with_overlap = df_phone_trunc_with_overlap[remove_overlap_mask]